In [ ]:
%matplotlib widget
from spectraclass.gui.spatial.aviris.manager import AvirisDatasetManager

widget = AvirisDatasetManager( cache_dir = "/adapt/nobackup/projects/ilab/cache",
                               data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/" )
widget.gui()